<a href="https://colab.research.google.com/github/gabrieladamasceno/Attacks_5G/blob/main/Attacks_5G_Aprendizado_Federado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Carregamento e Preparação do Dataset




In [2]:
from google.colab import drive
import pandas as pd

# Acessar dataset
drive.mount('/content/gdrive')

# Carregar o dataset
encoded = pd.read_csv("/content/gdrive/MyDrive/Datasets/Attacks/Encoded/Encoded.csv")

# Verificar as primeiras linhas do dataset
encoded.head()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,Unnamed: 0,Seq,Dur,RunTime,Mean,Sum,Min,Max,sTos,dTos,...,54,af11,af12,af41,cs0,cs4,cs6,cs7,ef,nan
0,0,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2,3,4.998020,4.998020,4.998020,4.998020,4.998020,4.998020,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,3,4,4.998037,4.998037,4.998037,4.998037,4.998037,4.998037,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,4,5,4.999453,4.999453,4.999453,4.999453,4.999453,4.999453,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [3]:
print(encoded.dtypes)
print(encoded.columns)

Unnamed: 0      int64
Seq             int64
Dur           float64
RunTime       float64
Mean          float64
               ...   
cs4           float64
cs6           float64
cs7           float64
ef            float64
nan           float64
Length: 96, dtype: object
Index(['Unnamed: 0', 'Seq', 'Dur', 'RunTime', 'Mean', 'Sum', 'Min', 'Max',
       'sTos', 'dTos', 'sTtl', 'dTtl', 'sHops', 'dHops', 'TotPkts', 'SrcPkts',
       'DstPkts', 'TotBytes', 'SrcBytes', 'DstBytes', 'Offset', 'sMeanPktSz',
       'dMeanPktSz', 'Load', 'SrcLoad', 'DstLoad', 'Loss', 'SrcLoss',
       'DstLoss', 'pLoss', 'SrcGap', 'DstGap', 'Rate', 'SrcRate', 'DstRate',
       'SrcWin', 'DstWin', 'sVid', 'dVid', 'SrcTCPBase', 'DstTCPBase',
       'TcpRtt', 'SynAck', 'AckDat', 'Label', 'Attack Type', 'Attack Tool',
       ' *        ', ' *    V   ', ' *    f   ', ' e        ', ' e    f   ',
       ' e &      ', ' e *      ', ' e d      ', ' e g      ', ' e i      ',
       ' e r      ', ' e s      ', ' eU       ', 'e 

# Tratamento dos Dados

In [4]:
from sklearn.preprocessing import LabelEncoder

# Codificar a coluna 'Label'
label_encoder = LabelEncoder()
encoded['Label'] = label_encoder.fit_transform(encoded['Label'])

In [5]:
import torch

# Separar features e labels
data = encoded[['Seq']]  # Usar 'Seq' como feature
labels = encoded['Label']  # 'Label' como rótulo

# Converter para NumPy arrays
data = data.values
labels = labels.values

# Converter para tensores PyTorch
data = torch.tensor(data, dtype=torch.float32)
labels = torch.tensor(labels, dtype=torch.long)

In [6]:
from sklearn.model_selection import train_test_split

# Dividir o dataset em treinamento e teste
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Converter para tensores PyTorch
data_train = torch.tensor(data_train, dtype=torch.float32)
labels_train = torch.tensor(labels_train, dtype=torch.long)
data_test = torch.tensor(data_test, dtype=torch.float32)
labels_test = torch.tensor(labels_test, dtype=torch.long)

<ipython-input-6-be25bb0f315c>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data_train = torch.tensor(data_train, dtype=torch.float32)
<ipython-input-6-be25bb0f315c>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_train = torch.tensor(labels_train, dtype=torch.long)
<ipython-input-6-be25bb0f315c>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data_test = torch.tensor(data_test, dtype=torch.float32)
<ipython-input-6-be25bb0f315c>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().d

In [7]:
from torch.utils.data import DataLoader, TensorDataset

# Criar datasets PyTorch
train_dataset = TensorDataset(data_train, labels_train)
test_dataset = TensorDataset(data_test, labels_test)

# Criar DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Definir o Modelo para Classificação de Ataques

In [8]:
import torch.nn as nn
import torch.optim as optim

# Definir o modelo
class AttackModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(AttackModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Inicializar o modelo, critério de perda e otimizador
input_size = data.shape[1]
hidden_size = 50
output_size = len(label_encoder.classes_)  # Número de classes

model = AttackModel(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Função de treinamento local
def train_local(model, train_loader, criterion, optimizer, epochs=1):
    model.train()
    for epoch in range(epochs):
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
    return model.state_dict()

In [9]:
def aggregate_models(global_model, client_models):
    global_dict = global_model.state_dict()

    # Inicializar um dicionário para média dos parâmetros
    new_dict = {}
    for key in global_dict:
        new_dict[key] = torch.zeros_like(global_dict[key])

    # Soma dos parâmetros dos modelos dos clientes
    for model_dict in client_models:
        for key in global_dict:
            new_dict[key] += model_dict[key]

    # Média dos parâmetros
    for key in new_dict:
        new_dict[key] /= len(client_models)

    # Atualizar o modelo global com os parâmetros agregados
    global_model.load_state_dict(new_dict)
    return global_model

# Treinamento Local em Cada Cliente

In [17]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Função para simular treinamento federado
def federated_learning(global_model, train_loader, criterion, optimizer, epochs, num_clients):
    # Armazenar modelos dos clientes
    client_models = []

    for client in range(num_clients):
        # Inicializar o modelo do cliente com os parâmetros do modelo global
        client_model = AttackModel(input_size, hidden_size, output_size)
        client_model.load_state_dict(global_model.state_dict())

        # Treinar o modelo do cliente
        client_optimizer = optim.SGD(client_model.parameters(), lr=0.01)
        client_model_params = train_local(client_model, train_loader, criterion, client_optimizer, epochs)

        # Armazenar o modelo treinado
        client_models.append(client_model.state_dict())

    # Agregar os modelos dos clientes no servidor central
    global_model = aggregate_models(global_model, client_models)
    return global_model

# Inicializar o modelo global
global_model = AttackModel(input_size, hidden_size, output_size)

# Configurar o critério de perda e o otimizador global
criterion = nn.CrossEntropyLoss()

# Agregação Federada

In [20]:
# Simular o treinamento federado
num_clients = 5  # Número de clientes
epochs = 5  # Número de épocas para treinamento local

# Dividir o dataset para clientes
clients_data = [TensorDataset(data_train, labels_train) for _ in range(num_clients)]
clients_loaders = [DataLoader(d, batch_size=32, shuffle=True) for d in clients_data]

# Treinamento federado
global_model = federated_learning(global_model, clients_loaders[0], criterion, optim.SGD(global_model.parameters(), lr=0.01), epochs, num_clients)

# Testar o modelo global
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    accuracy = correct / total
    return accuracy

# Testar o modelo global
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
accuracy = test_model(global_model, test_loader)
print(f'Accuracy of the global model: {accuracy * 100:.2f}%')

Accuracy of the global model: 60.61%
